# Prédiction du Comportement des Donneurs de Sang

Projet de fin d'année – ENSIAS BI&A


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# Suppression des avertissements
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Librairies de preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import (classification_report, accuracy_score, confusion_matrix, 
                           roc_auc_score, roc_curve, precision_recall_curve,
                           f1_score, precision_score, recall_score)

# Modèles de Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

print("✅ Toutes les bibliothèques ont été importées avec succès!")
print(f"📅 Session démarrée le: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
def load_and_inspect_data(file_path):
    try:
        df = pd.read_excel(file_path)
        print(f"✅ Dataset chargé avec succès!")
        print(f"📊 Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes\n")
        
        # Affichage des informations de base
        print("🔍 APERÇU DES DONNÉES:")
        print("=" * 50)
        display(df.head())
        
        print("\n📈 INFORMATIONS GÉNÉRALES:")
        print("=" * 50)
        print(df.info())
        
        print("\n📊 STATISTIQUES DESCRIPTIVES:")
        print("=" * 50)
        display(df.describe(include='all'))
        
        print("\n❌ VALEURS MANQUANTES:")
        print("=" * 50)
        missing_data = df.isnull().sum()
        missing_percent = (missing_data / len(df)) * 100
        missing_df = pd.DataFrame({
            'Colonne': missing_data.index,
            'Valeurs manquantes': missing_data.values,
            'Pourcentage': missing_percent.values
        }).sort_values('Valeurs manquantes', ascending=False)
        
        display(missing_df[missing_df['Valeurs manquantes'] > 0])
        
        return df
        
    except Exception as e:
        print(f"❌ Erreur lors du chargement: {e}")
        return None

# Chargement des données
df = load_and_inspect_data("base1_nettoyee.xlsx")

#  NETTOYAGE ET PRÉPROCESSING AVANCÉ DES DONNÉES

In [ ]:
def advanced_data_cleaning(df):
    
    df_clean = df.copy()
    
    print("🧹 NETTOYAGE DES DONNÉES EN COURS...")
    print("=" * 50)
    
    # 1. Suppression des doublons
    initial_rows = len(df_clean)
    df_clean.drop_duplicates(inplace=True)
    duplicates_removed = initial_rows - len(df_clean)
    print(f"🔄 Doublons supprimés: {duplicates_removed}")
    
    # 2. Gestion des valeurs manquantes
    missing_before = df_clean.isnull().sum().sum()
    df_clean.dropna(inplace=True)
    missing_after = df_clean.isnull().sum().sum()
    print(f"❌ Valeurs manquantes supprimées: {missing_before - missing_after}")
    
    # 3. Normalisation des noms de villes (avec gestion d'erreurs)
    if 'Ville  المدينة' in df_clean.columns:
        ville_mapping = {
            'RABAT': 'Rabat', 'rabat': 'Rabat',
            'Casa': 'Casablanca', 'casa': 'Casablanca', 'CASA': 'Casablanca',
            'AGADIR': 'Agadir', 'agadir': 'Agadir',
            'FES': 'Fès', 'fes': 'Fès', 'Fes': 'Fès',
            'MARRAKECH': 'Marrakech', 'marrakech': 'Marrakech',
            'TANGER': 'Tanger', 'tanger': 'Tanger'
        }
        df_clean['Ville  المدينة'] = df_clean['Ville  المدينة'].replace(ville_mapping)
        print(f"🏙️ Noms de villes normalisés")
    
    # 4. Détection et traitement des outliers pour l'âge
    if 'âge' in df_clean.columns:
        Q1 = df_clean['âge'].quantile(0.25)
        Q3 = df_clean['âge'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers_before = len(df_clean)
        df_clean = df_clean[(df_clean['âge'] >= lower_bound) & (df_clean['âge'] <= upper_bound)]
        outliers_removed = outliers_before - len(df_clean)
        print(f"📊 Outliers d'âge supprimés: {outliers_removed}")
    
    print(f"✅ Nettoyage terminé. Taille finale: {df_clean.shape}")
    return df_clean

# Application du nettoyage
df_clean = advanced_data_cleaning(df)

# ENCODAGE INTELLIGENT DES VARIABLES CATÉGORIELLES

In [ ]:
def smart_encoding(df):
   
    df_encoded = df.copy()
    encoders_dict = {}
    
    print("🔢 ENCODAGE DES VARIABLES CATÉGORIELLES:")
    print("=" * 50)
    
    categorical_columns = df_encoded.select_dtypes(include=['object']).columns
    
    for col in categorical_columns:
        print(f"🔤 Encodage de '{col}'...")
        
        # Vérification du nombre de catégories uniques
        unique_values = df_encoded[col].nunique()
        print(f"   → {unique_values} valeurs uniques")
        
        # Encodage avec LabelEncoder
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        encoders_dict[col] = le
        
        # Affichage du mapping pour les colonnes importantes
        if unique_values <= 10:
            mapping = dict(zip(le.classes_, le.transform(le.classes_)))
            print(f"   → Mapping: {mapping}")
    
    print(f"✅ Encodage terminé pour {len(categorical_columns)} colonnes")
    return df_encoded, encoders_dict

# Application de l'encodage
df_encoded, label_encoders = smart_encoding(df_clean)

# Vérification du résultat
print("\n📋 INFORMATIONS POST-ENCODAGE:")
print("=" * 50)
print(df_encoded.info())

# ANALYSE EXPLORATOIRE APPROFONDIE DES DONNÉES

In [ ]:
def comprehensive_eda(df):
   
    print("📊 ANALYSE EXPLORATOIRE DES DONNÉES")
    print("=" * 50)
    
    # 1. Distribution de l'âge
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    sns.histplot(data=df, x='âge', kde=True, bins=30)
    plt.title("Distribution des âges", fontsize=14, fontweight='bold')
    plt.xlabel("Âge")
    plt.ylabel("Fréquence")
    
    # 2. Répartition par sexe
    plt.subplot(1, 3, 2)
    sex_counts = df['sexe'].value_counts()
    plt.pie(sex_counts.values, labels=['Homme', 'Femme'], autopct='%1.1f%%', startangle=90)
    plt.title("Répartition par sexe", fontsize=14, fontweight='bold')
    
    # 3. Variable cible
    plt.subplot(1, 3, 3)
    target_counts = df['Don_de_sang'].value_counts()
    colors = ['lightcoral', 'lightblue']
    plt.pie(target_counts.values, labels=['Non-donneur', 'Donneur'], 
            autopct='%1.1f%%', colors=colors, startangle=90)
    plt.title("Répartition des donneurs", fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # 4. Distribution par région
    plt.figure(figsize=(12, 8))
    region_counts = df['région'].value_counts().head(15)
    sns.barplot(x=region_counts.values, y=region_counts.index, palette='viridis')
    plt.title("Top 15 des régions par nombre de participants", fontsize=16, fontweight='bold')
    plt.xlabel("Nombre de participants")
    plt.ylabel("Région")
    plt.show()
    
    # 5. Analyse des corrélations
    plt.figure(figsize=(12, 10))
    correlation_matrix = df.select_dtypes(include=[np.number]).corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5)
    plt.title("Matrice de corrélation", fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # 6. Analyse bivariée avec la variable cible
    plt.figure(figsize=(15, 10))
    
    # Âge vs Don de sang
    plt.subplot(2, 3, 1)
    sns.boxplot(data=df, x='Don_de_sang', y='âge')
    plt.title("Âge par statut de don")
    plt.xlabel("Don de sang (0=Non, 1=Oui)")
    
    # Sexe vs Don de sang
    plt.subplot(2, 3, 2)
    ct = pd.crosstab(df['sexe'], df['Don_de_sang'], normalize='index') * 100
    ct.plot(kind='bar', ax=plt.gca())
    plt.title("Taux de don par sexe (%)")
    plt.xlabel("Sexe")
    plt.ylabel("Pourcentage")
    plt.legend(['Non-donneur', 'Donneur'])
    plt.xticks(rotation=0)
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques descriptives par groupe
    print("\n📈 STATISTIQUES PAR GROUPE:")
    print("=" * 50)
    stats_by_group = df.groupby('Don_de_sang')['âge'].describe()
    print(stats_by_group)

# Exécution de l'EDA
comprehensive_eda(df_encoded)

# PRÉPARATION DES DONNÉES POUR LA MODÉLISATION

In [ ]:
def prepare_modeling_data(df, target_column='Don_de_sang', test_size=0.2, random_state=42):
    
    print("🎯 PRÉPARATION DES DONNÉES POUR LA MODÉLISATION")
    print("=" * 50)
    
    # Séparation des features et de la cible
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    
    print(f"📊 Nombre de features: {X.shape[1]}")
    print(f"🎯 Distribution de la variable cible:")
    print(y.value_counts(normalize=True))
    
    # Division train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    print(f"\n✅ Division des données:")
    print(f"   → Training set: {X_train.shape[0]} échantillons")
    print(f"   → Test set: {X_test.shape[0]} échantillons")
    
    # Normalisation des données (optionnel mais recommandé)
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled, scaler

# Préparation des données
X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled, scaler = prepare_modeling_data(df_encoded)


# MODÉLISATION MACHINE LEARNING AVANCÉE


In [ ]:
class MLModelEvaluator:
   
    
    def __init__(self):
        self.results = {}
        self.models = {}
    
    def evaluate_model(self, name, model, X_train, X_test, y_train, y_test, cv_folds=5):
        """
        Évalue un modèle de manière complète
        """
        print(f"\n🤖 ÉVALUATION DU MODÈLE: {name}")
        print("=" * 60)
        
        # Entraînement
        start_time = datetime.now()
        model.fit(X_train, y_train)
        training_time = (datetime.now() - start_time).total_seconds()
        
        # Prédictions
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Métriques de base
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Cross-validation
        cv_scores = cross_val_score(model, X_train, y_train, cv=cv_folds, scoring='accuracy')
        
        # AUC-ROC si probabilités disponibles
        auc_roc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None
        
        # Stockage des résultats
        self.results[name] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'auc_roc': auc_roc,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'training_time': training_time,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        self.models[name] = model
        
        # Affichage des résultats
        print(f"📊 Accuracy: {accuracy:.4f}")
        print(f"🎯 Precision: {precision:.4f}")
        print(f"🔍 Recall: {recall:.4f}")
        print(f"⚖️ F1-Score: {f1:.4f}")
        if auc_roc:
            print(f"📈 AUC-ROC: {auc_roc:.4f}")
        print(f"🔄 CV Score: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
        print(f"⏱️ Training Time: {training_time:.2f}s")
        
        print(f"\n📋 Rapport de classification:")
        print(classification_report(y_test, y_pred))
        
        return model
    
    def plot_confusion_matrices(self, y_test):
        
        n_models = len(self.results)
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        for i, (name, results) in enumerate(self.results.items()):
            if i < len(axes):
                cm = confusion_matrix(y_test, results['predictions'])
                sns.heatmap(cm, annot=True, fmt='d', ax=axes[i], cmap='Blues')
                axes[i].set_title(f'Matrice de confusion - {name}')
                axes[i].set_xlabel('Prédictions')
                axes[i].set_ylabel('Valeurs réelles')
        
        # Masquer les axes inutilisés
        for j in range(i+1, len(axes)):
            axes[j].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    
    def plot_roc_curves(self, y_test):
       
        plt.figure(figsize=(12, 8))
        
        for name, results in self.results.items():
            if results['probabilities'] is not None:
                fpr, tpr, _ = roc_curve(y_test, results['probabilities'])
                auc = results['auc_roc']
                plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=2)
        
        plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
        plt.xlabel('Taux de Faux Positifs')
        plt.ylabel('Taux de Vrais Positifs')
        plt.title('Courbes ROC - Comparaison des modèles')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
    
    def get_results_summary(self):
       
        results_df = pd.DataFrame(self.results).T
        results_df = results_df.round(4)
        return results_df.sort_values('accuracy', ascending=False)

# Initialisation de l'évaluateur
evaluator = MLModelEvaluator()

# ENTRAÎNEMENT ET ÉVALUATION DES MODÈLES

In [ ]:
# 1. Régression Logistique
print("🚀 DÉMARRAGE DE LA MODÉLISATION")
print("=" * 60)

lr_model = LogisticRegression(max_iter=1000, random_state=42)
evaluator.evaluate_model("Régression Logistique", lr_model, X_train_scaled, X_test_scaled, y_train, y_test)

# 2. K-Nearest Neighbors
knn_model = KNeighborsClassifier(n_neighbors=5)
evaluator.evaluate_model("K-Nearest Neighbors", knn_model, X_train_scaled, X_test_scaled, y_train, y_test)

# 3. Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
evaluator.evaluate_model("Random Forest", rf_model, X_train, X_test, y_train, y_test)

# 4. Support Vector Machine
svm_model = SVC(probability=True, random_state=42)
evaluator.evaluate_model("Support Vector Machine", svm_model, X_train_scaled, X_test_scaled, y_train, y_test)

# 5. XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
evaluator.evaluate_model("XGBoost", xgb_model, X_train, X_test, y_train, y_test)

# 6. LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42, verbose=-1)
evaluator.evaluate_model("LightGBM", lgb_model, X_train, X_test, y_train, y_test)

# 7. CatBoost
cat_model = CatBoostClassifier(verbose=0, random_state=42)
evaluator.evaluate_model("CatBoost", cat_model, X_train, X_test, y_train, y_test)



# ANALYSE COMPARATIVE DES RÉSULTATS

In [ ]:
print("\n🏆 RÉSUMÉ COMPARATIF DES PERFORMANCES")
print("=" * 60)

# Tableau comparatif
results_summary = evaluator.get_results_summary()
print(results_summary)

# Visualisations comparatives
evaluator.plot_confusion_matrices(y_test)
evaluator.plot_roc_curves(y_test)

# Graphique comparatif des métriques principales
plt.figure(figsize=(15, 10))

metrics = ['accuracy', 'precision', 'recall', 'f1_score']
model_names = list(evaluator.results.keys())

for i, metric in enumerate(metrics):
    plt.subplot(2, 2, i+1)
    values = [evaluator.results[model][metric] for model in model_names]
    bars = plt.bar(model_names, values, alpha=0.8)
    plt.title(f'Comparaison - {metric.upper()}')
    plt.ylabel('Score')
    plt.xticks(rotation=45)
    
    # Ajouter les valeurs sur les barres
    for bar, value in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

#  OPTIMISATION DU MEILLEUR MODÈLE

In [ ]:
def optimize_best_model(evaluator, X_train, X_test, y_train, y_test):
   
    # Identifier le meilleur modèle
    best_model_name = evaluator.get_results_summary().index[0]
    print(f"\n🎯 OPTIMISATION DU MEILLEUR MODÈLE: {best_model_name}")
    print("=" * 60)
    
    if "XGBoost" in best_model_name:
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
        model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
        
    elif "Random Forest" in best_model_name:
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10]
        }
        model = RandomForestClassifier(random_state=42)
    
    else:
        print("Optimisation non implémentée pour ce modèle")
        return None
    
    # GridSearchCV
    grid_search = GridSearchCV(
        model, param_grid, cv=5, scoring='accuracy', 
        n_jobs=-1, verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    
    print(f"🏆 Meilleurs paramètres: {grid_search.best_params_}")
    print(f"📊 Meilleur score CV: {grid_search.best_score_:.4f}")
    
    # Évaluation du modèle optimisé
    optimized_model = grid_search.best_estimator_
    evaluator.evaluate_model(f"{best_model_name} (Optimisé)", 
                           optimized_model, X_train, X_test, y_train, y_test)
    
    return optimized_model

# Optimisation du meilleur modèle
best_optimized_model = optimize_best_model(evaluator, X_train, X_test, y_train, y_test)

# ANALYSE DE L'IMPORTANCE DES VARIABLES

In [ ]:
def analyze_feature_importance(model, feature_names, model_name):
    
    if hasattr(model, 'feature_importances_'):
        print(f"\n🔍 IMPORTANCE DES VARIABLES - {model_name}")
        print("=" * 60)
        
        importance_df = pd.DataFrame({
            'Variable': feature_names,
            'Importance': model.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        print(importance_df.head(10))
        
        # Visualisation
        plt.figure(figsize=(12, 8))
        top_features = importance_df.head(15)
        sns.barplot(data=top_features, x='Importance', y='Variable')
        plt.title(f'Top 15 Variables les plus importantes - {model_name}')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.show()
        
        return importance_df
    else:
        print(f"Le modèle {model_name} ne supporte pas l'analyse d'importance des variables")
        return None

# Analyse pour les modèles basés sur les arbres
feature_names = X_train.columns
for name, model in evaluator.models.items():
    if hasattr(model, 'feature_importances_'):
        analyze_feature_importance(model, feature_names, name)
